1\. rozglądam sie po systemie, sprawdzam użytkowników, bazy

In [ ]:
use master;
select [name] from master.sys.databases;
select [name] from master.sys.sql_logins;

2\. zakładam bazę

In [75]:
/*
IF EXISTS (SELECT [name] FROM master.sys.sysdatabases WHERE name = 'DB_OKNO')
    BEGIN
    PRINT 'baza DB_OKNO istnieje';    
    DROP DATABASE DB_OKNO;
    PRINT 'baza usunieta';
    END
*/

DROP DATABASE IF EXISTS DB_OKNO ;
go

CREATE DATABASE DB_OKNO 
/*    ON PRIMARY (
        name=okno,
        filename='/var/opt/mssql/data/okno.mdf',
        size=20MB,
        maxsize=50MB,
        filegrowth=10MB
    )
    LOG ON (
        name=okno_log,
        filename='/var/opt/mssql/data/okno.ldf',
        size=20MB,
        maxsize=50MB,
        filegrowth=10MB
    )  */
COLLATE POLISH_100_CI_AS_SC_UTF8
GO
    

    

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.769

dodaję tabele   \*\*\* SŁOWNIKÓW \*\*\*

In [76]:
use DB_OKNO
GO

/* *** slowniki *** */
DROP TABLE IF EXISTS icon;
DROP TABLE IF EXISTS lang;
DROP TABLE IF EXISTS issue;
DROP TABLE IF EXISTS code;
DROP TABLE IF EXISTS category;

go
CREATE TABLE category (
    ID_category DECIMAL(10) NOT NULL ,
    category_name nvarchar(32)

    PRIMARY KEY ( ID_category )
);
go


INSERT INTO category VALUES( 10000 , 'Kosiarki'); 
INSERT INTO category VALUES( 10001 , '- Kosiarki spalinowe'); 
INSERT INTO category VALUES( 10002 , '- Kosiarki elektryczne'); 
INSERT INTO category VALUES( 20000 , 'Ślimacznice'); 



CREATE TABLE code (
    ID_code BIGINT NOT NULL IDENTITY,
    code_string varchar(32),
    EAN CHAR(13), -- po EAN (w tym rozwiazaniu) nie wyszukujemy, są EAN zaczynające się od 0 dlatego CHAR a nie BIGINT/DECIMAL(13)
    active BIT NOT NULL DEFAULT 1,
    category DECIMAL(10), -- kategorie 1.000, 2.000  podkategorie 1.001 1.002 ...   taka notacja jest narzucona przez zewnetrzna bazę

    PRIMARY KEY ( ID_code ),
    FOREIGN KEY ( category ) REFERENCES category( ID_category )
    ON DELETE CASCADE
    ON UPDATE CASCADE
);
go


INSERT INTO code ( code_string ) VALUES ( '80C051' );  
INSERT INTO code ( code_string ) VALUES ( '80C052' );

INSERT INTO code ( code_string ) VALUES ( '0602'); 
INSERT INTO code ( code_string ) VALUES ( '0603');  

INSERT INTO code ( code_string ) VALUES ( 'DED780911');  
INSERT INTO code ( code_string ) VALUES ( 'DED780912');  
INSERT INTO code ( code_string ) VALUES ( 'DED780920');




CREATE TABLE issue (
    ID_issue BIGINT NOT NULL IDENTITY ,
    issue_name varchar(255) NOT NULL 

    PRIMARY KEY ( ID_issue )
);
go


INSERT INTO issue ( issue_name ) VALUES ( '2022.10');
INSERT INTO issue ( issue_name ) VALUES ( '2022.11.Garden');
INSERT INTO issue ( issue_name ) VALUES ( '2023.01');



CREATE TABLE lang (
    ID_lang BIGINT NOT NULL,
    lang_code char(2) NOT NULL,
    currency  nchar(2) NOT NULL,
    zeros   int NOT NULL,
    table_header nvarchar(255),

    PRIMARY KEY ( ID_lang )
);
go



    INSERT INTO lang VALUES (1,'PL','zł',2,'nr. kat.|%|cena|promocja');
    INSERT INTO lang VALUES (2,'CZ','Kč',1,'kat. č.|popis|cena|akční cena');
    INSERT INTO lang VALUES (3,'EN','€',2, 'cat. no.|description|price|promo price');
    INSERT INTO lang VALUES (4,'HU','Ft',0,'cikkszám|leírás|ár|akciós ár');
    -- ****
    INSERT INTO lang VALUES (15,'UA','€',2,'Код|опис|ціна|акційна ціна');





CREATE TABLE icon (
    ID_icon BIGINT NOT NULL IDENTITY,
    icon_name nvarchar(255) NOT NULL UNIQUE,
    -- edit by dept.
    tech  BIT DEFAULT(0),
    promo BIT DEFAULT(0),
    -- target
    paper BIT DEFAULT(0),
    catal BIT DEFAULT(0),
    shipping BIT DEFAULT(0),
    rack BIT DEFAULT(0),

    PRIMARY KEY ( ID_icon )
);
go    

    INSERT INTO icon ( icon_name, tech, rack  ) VALUES ('Moc silnika elektycznego',1,1);
    INSERT INTO icon ( icon_name, tech, rack  ) VALUES ('Moc silnika spalinowego' ,1,1);








Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.279

tworzę tabelę indd\_name wraz z procedurą

In [174]:
use DB_OKNO;
GO

DROP TABLE IF EXISTS indd_names;
go

CREATE TABLE indd_names (
    ID_code BIGINT NOT NULL,
    ID_lang BIGINT NOT NULL,

    i_name nvarchar(255),
    i_description nvarchar(max),
    
    PRIMARY KEY ( ID_code , ID_lang )
);
go  


-- INSERT PROCEDURE 
CREATE OR ALTER PROCEDURE add_names
    @code_string varchar(32),
    @lang_id BIGINT,
    @i_name nvarchar(255),
    @i_description nvarchar(max)
AS
BEGIN
--   PRINT @code_string; PRINT @lang_id;  PRINT @i_name; PRINT @i_description; 

    DECLARE @cID BIGINT = NULL;
    SET @cID =  ( SELECT ID_code FROM code AS C WHERE C.code_string = @code_string );
    IF @cID IS NOT NULL 
        BEGIN 
             -- PRINT 'not NULL !';
             -- PRINT  ( CONCAT ( CONCAT ( @code_string , ' : ')  , @cID )) ;
            IF NOT EXISTS ( SELECT ID_code FROM indd_names AS N WHERE ( N.ID_code=@cID AND N.ID_lang = @lang_id) ) 
                BEGIN 
                    PRINT 'INSERT';
                    INSERT INTO indd_names (ID_code , ID_lang , i_name , i_description ) VALUES ( @cID, @lang_id , @i_name , @i_description ); 
                END
            ELSE
                BEGIN 
                    PRINT 'UPDATE';
                    UPDATE indd_names  SET  i_name = @i_name , i_description = @i_description  WHERE ( ID_code = @cID AND ID_lang = @lang_id ) ; 
                END 
        END
END


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.070

In [173]:
-- uruchamiam procedure wstawiania tresci

USE DB_OKNO;
go

execute add_names  '0603' , 1 , 'Klucz', 'CRV - first insert' ;
go

execute add_names  '0603' , 1 , 'Klucz', 'CRV - second insert' ;
go

execute add_names  '80C051' , 1 , 'Pompa', 'do wody brudnej' ;
go

execute add_names  ' -- Fake -- ' , 1 , ' ??? ', ' ??? ' ;
go

Commands completed successfully.

0603

1

Klucz

CRV - first insert

...

-- Fake --

1

???

???

Total execution time: 00:00:00.047